# Hu+BP

## 0.生成Hu值

In [ ]:
#Hu不变矩代码 python+opencv
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

path='./data/Hu_data/'

lbe = LabelEncoder()
mms = MinMaxScaler(feature_range=(0, 1))

def Hu_moments(img):
    '''
    opencv_python自带求矩以及不变矩的函数
    :param img: 灰度图像，对于二值图像来说就只有两个灰度0和255
    :return: 返回以10为底对数化后的hu不变矩
    '''
    moments = cv2.moments(img)  # 返回的是一个字典，三阶及以下的几何矩（mpq）、中心矩(mupq)和归一化的矩(nupq)
    humoments = cv2.HuMoments(moments)  # 根据几何矩（mpq）、中心矩(mupq)和归一化的矩(nupq)计算出hu不变矩
    # 因为直接计算出来的矩可能很小或者很大，因此取对数好比较,这里的对数底数为e,通过对数除法的性质将其转换为以10为底的对数
    humoment = (np.log(np.abs(humoments))) / np.log(10)
    humoment = np.reshape(humoment,(1,7))[0]
    return humoment
if __name__ == '__main__':
    hu=[]
    img_col=[]
    file_list = os.listdir(path)
    for img_name in file_list:
        img = cv2.imread(path+img_name, 0) #读入图片
        humoments = Hu_moments(img)
        img_col.append(img_name.split('_')[0])
        hu.append(humoments)
    hu=pd.DataFrame(hu)
    hu[7]=img_col
    
    
    hu[7] = lbe.fit_transform(hu[7])
    hu.sort_values(by=[7],inplace=True)
    hu.to_csv('./data/hu.csv',index=False,header=None)

## 1.BP神经网络

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn.functional as F

path='./data/'

In [ ]:
def Z_ScoreNormalization(x):            #归一化
    mu=np.average(x)
    sigma=np.std(x)
    x = (x - mu) / sigma
    return x

def norm_all(x):            #实现了每列归一化
    x=np.array(x)[:,:-1]
    h,w=x.shape
    # print(h,w)
    x=x.T
    rlt=[]
    for i in range(w):
        # print('guagua',x[i])
        tmp=Z_ScoreNormalization(x[i])
        rlt.append(tmp)
    rlt=np.array(rlt).T
    return rlt

In [ ]:
class DealDataset(Dataset):
    def __init__(self):
        xy = np.loadtxt(path+'./hu.csv', delimiter=',', dtype=np.float32)  # 使用numpy读取数据
        lab=xy[:,-1].reshape(-1,1)
        xy=norm_all(xy)
        xy=np.concatenate((xy,lab),1)
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, -1]).long()
        self.len = xy.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
# Hu数据加载、划分数据集
data = DealDataset()
train_set, test_set = random_split(data,lengths=[720, 180])  #300个样本，按8:2划分训练集和验证集

train_data = DataLoader(train_set, batch_size=4, shuffle=False)  # 训练数据
test_data = DataLoader(test_set, batch_size=4, shuffle=False)  # 测试数据

In [ ]:
# 模型定义
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linear1 = torch.nn.Linear(7,5)
        self.linear2 = torch.nn.Linear(5,3)

    def forward(self,x):
        x = x.view(-1,7) # 改变张量形状。把输入展开成若干行，7列
        x = F.leaky_relu(self.linear1(x))
        return self.linear2(x) #最后一层不做激活，因为下一步输入到交叉损失函数中，交叉熵包含了激活层
    
model = Net()
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #, momentum= 0.5

In [ ]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_data,0):
        inputs,labels=data
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        if batch_idx%100==99:
            print('[%d,%5d] loss:%.3f'%(epoch+1,batch_idx+1,running_loss/100))
            running_loss=0.0

In [ ]:
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_data:
            images,labels=data
            images,labels=images.to(device),labels.to(device)
            outputs=model(images)
            _,predicted=torch.max(outputs.data,dim=1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()
    print('Accuracy on test set:%d %%'%(100*correct/total))

In [ ]:
if __name__=='__main__':
    for epoch in range(100):
        train(epoch)
        test()